# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Final Project: Part 1 - Exploratory Analysis


We are going to do some exploratory analysis on the data to see what is available. This data is from a [Kaggle competition](https://www.kaggle.com/c/home-credit-default-risk/data). Although we are not going to be entering the competition, we will come up with something reasonable. See the associated Final Project note in the course OneNote on the acceptance criteria and requirements. 

The final project consists of three SQLite databases. The databases below are identical but sampled as needed. Note that databases are read-only so you should not try to create or alter the contents. You may create another database in your directory.

* Train: Used for full model training, as needed.
* Dev: Used for quick prototyping, validation, etc.
* Test: Used to grade and evaluate your work (Note: Used when grading only).

They all have the same schema, shown below. Note that there is no "application_test". 

![alt text](home_credit.png)


In [1]:
require './assignment_lib.rb'

dir = "/home/dataset"
$dev_db = SQLite3::Database.new "#{dir}/credit_risk_data_dev.db", results_as_hash: true, readonly: true
$train_db = SQLite3::Database.new "#{dir}/credit_risk_data_train.db", results_as_hash: true, readonly: true

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

#<SQLite3::Database:0x0000000001b46d28 @tracefunc=nil, @authorizer=nil, @encoding=nil, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=true, @type_translation=nil, @readonly=true>

## Getting to know the data
Here are some sample queries to show the tables and columns for a table. You are encourages to play around with the data and export some data to your favorite data analysis system.

In [4]:
sql = <<SQL
SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;"
SQL
     

$dev_db.execute(sql)

[{"name"=>"POS_CASH_balance", 0=>"POS_CASH_balance"}, {"name"=>"application_train", 0=>"application_train"}, {"name"=>"bureau", 0=>"bureau"}, {"name"=>"bureau_balance", 0=>"bureau_balance"}, {"name"=>"credit_card_balance", 0=>"credit_card_balance"}, {"name"=>"data_sample", 0=>"data_sample"}, {"name"=>"installments_payments", 0=>"installments_payments"}, {"name"=>"previous_application", 0=>"previous_application"}]

In [3]:
sql = "select * from application_train"
$dev_db.prepare(sql).columns

["SK_ID_CURR", "TARGET", "NAME_CONTRACT_TYPE", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", "NAME_TYPE_SUITE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "REGION_POPULATION_RELATIVE", "DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "OWN_CAR_AGE", "FLAG_MOBIL", "FLAG_EMP_PHONE", "FLAG_WORK_PHONE", "FLAG_CONT_MOBILE", "FLAG_PHONE", "FLAG_EMAIL", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS", "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY", "WEEKDAY_APPR_PROCESS_START", "HOUR_APPR_PROCESS_START", "REG_REGION_NOT_LIVE_REGION", "REG_REGION_NOT_WORK_REGION", "LIVE_REGION_NOT_WORK_REGION", "REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "LIVE_CITY_NOT_WORK_CITY", "ORGANIZATION_TYPE", "EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "APARTMENTS_AVG", "BASEMENTAREA_AVG", "YEARS_BEGINEXPLUATATION_AVG", "YEARS_BUILD_AVG", "COMMONAREA_AVG", "ELE

In [9]:
sql = "select * from previous_application"
$dev_db.prepare(sql).columns

["SK_ID_PREV", "SK_ID_CURR", "NAME_CONTRACT_TYPE", "AMT_ANNUITY", "AMT_APPLICATION", "AMT_CREDIT", "AMT_DOWN_PAYMENT", "AMT_GOODS_PRICE", "WEEKDAY_APPR_PROCESS_START", "HOUR_APPR_PROCESS_START", "FLAG_LAST_APPL_PER_CONTRACT", "NFLAG_LAST_APPL_IN_DAY", "RATE_DOWN_PAYMENT", "RATE_INTEREST_PRIMARY", "RATE_INTEREST_PRIVILEGED", "NAME_CASH_LOAN_PURPOSE", "NAME_CONTRACT_STATUS", "DAYS_DECISION", "NAME_PAYMENT_TYPE", "CODE_REJECT_REASON", "NAME_TYPE_SUITE", "NAME_CLIENT_TYPE", "NAME_GOODS_CATEGORY", "NAME_PORTFOLIO", "NAME_PRODUCT_TYPE", "CHANNEL_TYPE", "SELLERPLACE_AREA", "NAME_SELLER_INDUSTRY", "CNT_PAYMENT", "NAME_YIELD_GROUP", "PRODUCT_COMBINATION", "DAYS_FIRST_DRAWING", "DAYS_FIRST_DUE", "DAYS_LAST_DUE_1ST_VERSION", "DAYS_LAST_DUE", "DAYS_TERMINATION", "NFLAG_INSURED_ON_APPROVAL"]

## Question 1.1 (20 points)

Getting familiar with SQL, try counting the number of rows in the dev database and the train database. Return a single number from an SQL query.

Here is an example of how to run a query:

```ruby
def count_applications db
    db.execute("select * from application_train limit 10") do |row|
        puts row["TARGET"]
    end
end
```

In [5]:
def count_applications db
  # BEGIN YOUR CODE
  return db.execute("SELECT count(*) FROM application_train")[0]["count(*)"]
  #END YOUR CODE
end

:count_applications

In [6]:
def test_11()
  assert_equal 15334, count_applications($dev_db)
  assert_equal 276667, count_applications($train_db)
end

test_11()

## Begin Question 1.2 (10 points)

This dataset presents a classification problem in which we seek to predict positive (target = 1) applications versus negative (target = 0) applications. Each application is identified by ```SK_ID_CURR``` in a record of the table ```application_train```. Other tables join on the ```SK_ID_CURR``` field. 

Return the number of applications with target value 0 and 1 as a hash. Name the counts as "n". 

In [7]:
def target_distribution(db)
  # BEGIN YOUR CODE
  distribution = Hash[0, 0, 1, 0]
  db.execute("SELECT * FROM application_train") do |row|
    if(row["TARGET"] == 0)
      distribution[0] += 1
    end
    if(row["TARGET"] == 1)
      distribution[1] += 1
    end
  end    
  #END YOUR CODE
  return distribution
end

:target_distribution

In [8]:
# Tests for the counts
def test_12()
  dist = target_distribution($dev_db)
  assert_equal 14101, dist[0]
  assert_equal 1233, dist[1]
end
test_12()

## Begin Question 1.3 (10 points)

Write a function which, when provided a database and a name of a column with categorical values, returns a hash with the top 10 values, along with their counts. 


In [9]:
def feature_frequency db, fname
  frequency = Hash.new
  # BEGIN YOUR CODE
  db.execute("SELECT * FROM application_train") do |row|
    if(frequency.has_key?(row[fname]))
      frequency[row[fname]] += 1
    else
      frequency[row[fname]] = 1
    end
  end
  #END YOUR CODE
  return frequency
end


:feature_frequency

In [10]:
def test_13
  d_ct = feature_frequency $dev_db, "NAME_CONTRACT_TYPE"
  assert_equal 2, d_ct.size
  assert_equal 13867, d_ct["Cash loans"]
  assert_equal 1467, d_ct["Revolving loans"]
end

test_13()

## Begin Question 1.4 (10 points)

Count the number of positive (target = 1) and negative (target = 0) applications from ```application_train``` with exactly 4 records in ```previous_application```.

In [2]:
def num_with_prev(db)
  counts = Hash.new
  # BEGIN YOUR CODE
  sql = "SELECT application_train.target, COUNT(application_train.target) FROM application_train
         INNER JOIN (SELECT SK_ID_CURR, COUNT(SK_ID_CURR) AS count FROM previous_application
          GROUP BY SK_ID_CURR HAVING count=4) AS G
          ON application_train.SK_ID_CURR=G.SK_ID_CURR
         GROUP BY application_train.target"
  db.execute(sql).each do |row|
    counts[row[0]] = row[1]
  end   
  #END YOUR CODE
  return counts
end

:num_with_prev

In [12]:
$dev_db.execute("SELECT * FROM application_train limit 2")

[{"SK_ID_CURR"=>100002, "TARGET"=>1, "NAME_CONTRACT_TYPE"=>"Cash loans", "CODE_GENDER"=>"M", "FLAG_OWN_CAR"=>"N", "FLAG_OWN_REALTY"=>"Y", "CNT_CHILDREN"=>0, "AMT_INCOME_TOTAL"=>202500, "AMT_CREDIT"=>406597.5, "AMT_ANNUITY"=>24700.5, "AMT_GOODS_PRICE"=>351000, "NAME_TYPE_SUITE"=>"Unaccompanied", "NAME_INCOME_TYPE"=>"Working", "NAME_EDUCATION_TYPE"=>"Secondary / secondary special", "NAME_FAMILY_STATUS"=>"Single / not married", "NAME_HOUSING_TYPE"=>"House / apartment", "REGION_POPULATION_RELATIVE"=>0.018801, "DAYS_BIRTH"=>-9461, "DAYS_EMPLOYED"=>-637, "DAYS_REGISTRATION"=>-3648, "DAYS_ID_PUBLISH"=>-2120, "OWN_CAR_AGE"=>"", "FLAG_MOBIL"=>1, "FLAG_EMP_PHONE"=>1, "FLAG_WORK_PHONE"=>0, "FLAG_CONT_MOBILE"=>1, "FLAG_PHONE"=>1, "FLAG_EMAIL"=>0, "OCCUPATION_TYPE"=>"Laborers", "CNT_FAM_MEMBERS"=>1, "REGION_RATING_CLIENT"=>2, "REGION_RATING_CLIENT_W_CITY"=>2, "WEEKDAY_APPR_PROCESS_START"=>"WEDNESDAY", "HOUR_APPR_PROCESS_START"=>10, "REG_REGION_NOT_LIVE_REGION"=>0, "REG_REGION_NOT_WORK_REGION"=>0, "

In [11]:
$dev_db.execute("SELECT * FROM previous_application limit 2")

[{"SK_ID_PREV"=>1397919, "SK_ID_CURR"=>321676, "NAME_CONTRACT_TYPE"=>"Consumer loans", "AMT_ANNUITY"=>7654.86, "AMT_APPLICATION"=>53779.5, "AMT_CREDIT"=>57564, "AMT_DOWN_PAYMENT"=>0, "AMT_GOODS_PRICE"=>53779.5, "WEEKDAY_APPR_PROCESS_START"=>"SUNDAY", "HOUR_APPR_PROCESS_START"=>15, "FLAG_LAST_APPL_PER_CONTRACT"=>"Y", "NFLAG_LAST_APPL_IN_DAY"=>1, "RATE_DOWN_PAYMENT"=>0, "RATE_INTEREST_PRIMARY"=>"", "RATE_INTEREST_PRIVILEGED"=>"", "NAME_CASH_LOAN_PURPOSE"=>"XAP", "NAME_CONTRACT_STATUS"=>"Approved", "DAYS_DECISION"=>-408, "NAME_PAYMENT_TYPE"=>"Cash through the bank", "CODE_REJECT_REASON"=>"XAP", "NAME_TYPE_SUITE"=>"Unaccompanied", "NAME_CLIENT_TYPE"=>"New", "NAME_GOODS_CATEGORY"=>"Consumer Electronics", "NAME_PORTFOLIO"=>"POS", "NAME_PRODUCT_TYPE"=>"XNA", "CHANNEL_TYPE"=>"Country-wide", "SELLERPLACE_AREA"=>"200", "NAME_SELLER_INDUSTRY"=>"Consumer electronics", "CNT_PAYMENT"=>8, "NAME_YIELD_GROUP"=>"low_action", "PRODUCT_COMBINATION"=>"POS household without interest", "DAYS_FIRST_DRAWING"=>

In [3]:
def test_14()
  t = num_with_prev($dev_db)
  assert_true(t.has_key? 0)
  assert_true(t.has_key? 1)
end

test_14()